In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import shutil

### 计算图片平均大小

In [2]:
## 计算图片平均大小
path = r'C:/Users/DELL/Desktop/11.28/data'
for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    #print(img.size)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'C:/Users/DELL/Desktop/11.28/data'

## 读入数据集

In [15]:
metadata_df = pd.read_csv("Metadata.csv",encoding='ISO-8859-15', header = None, low_memory = False)
df = metadata_df.loc[181618:181622, 0:12]
df.columns = ['ID', 'URL', 'Designer', 'Company_Name', 'Date', 'Tags', 'Views', 'Likes', 'Saves', 'Comments','Attachments',"Title", "Description"]
df["Attachments"] = df['Attachments'].astype(int)
df

,ID,URL,Designer,Company_Name,Date,Tags,Views,Likes,Saves,Comments,Attachments,Title,Description
181618,4259251,https://dribbble.com/shots/4259251,MattLacourse,NaN,2018/2/22,brassmountain dailylogochallenge day7 ...,4,0,0,0,0,008 - Brass Mountain,"Let's see if you can find the small yet, in m..."
181619,4259252,https://dribbble.com/shots/4259252,Nick_Esposito,NaN,2018/2/22,caveman,14,0,0,0,0,(Really) Old Time Hockey,Fantasy hockey team idea.
181620,4259253,https://dribbble.com/shots/4259253,Amomo_0312,NaN,2018/2/22,illustration,134,23,0,6,0,A dream of opera,NaN
181621,4259254,https://dribbble.com/shots/4259254,JoshHeru,NaN,2018/2/22,flyer flyercharity flyernetworking f...,70,10,0,0,0,Flyer Event,NaN
181622,4259256,https://dribbble.com/shots/4259256,tc_timothy,NaN,2018/2/22,art blue flat hug illustration l...,239,32,0,0,0,To My Most Beloved Girl,I embrace the moon as if I embraced you.


In [2]:
## 从总文件夹中取出符合tag条件的图片 Step1 读入以及处理数据 ##

# read metadata.csv file
# metadata_col = ['id', 'link', 'creater', 'team_name', 'date', 'tags', 'views', 'likes', 'saves', 'comments','attachment',"title", "description"]
metadata_df = pd.read_csv("Metadata.csv",encoding='ISO-8859-15', header = None, low_memory = False)
metadata_df = metadata_df.loc[:, [0, 5]]
metadata_df = metadata_df.rename(columns={0:"id", 5: "tags"})
# get tags out and remove nan lines.
tags = metadata_df.loc[:,["id","tags"]]
tags = tags.dropna(axis=0,how='any')
tags.index = range(len(tags))
# 将tag列分成多列
tags_df = tags['tags'].str.split("   ")
tags_df = pd.DataFrame(tags_df)
for i in tags_df.index:
    x = len(tags_df["tags"][i])
    tags_df.loc[i,"tags"] = ",".join(tags_df['tags'][i][1:x - 1])
    
tags_df = tags_df["tags"].str.split(",", expand = True)

# 将新生成的多列tag和id合并
df = tags.drop("tags", axis = 1).join(tags_df)
df["id"] = df['id'].astype('str')

In [3]:
print(len(df))

226099


In [14]:
# delete all none entries in the dataframe.
df_temp = []
for i in range(0, len(df)):  
    df_temp.append([str(df.values[i,j]) for j in range(0, df.shape[1]) if df.values[i,j]!= None])
df_temp

[['4000000',
  '003',
  '30 days of ui',
  'colorful',
  'daily ui',
  'landing page',
  'music event',
  'ui'],
 ['4000002',
  'collection',
  'dunelm',
  'icon',
  'iconography',
  'line',
  'mills',
  'online',
  'set',
  'shop',
  'store'],
 ['4000004',
  'canvas',
  'canvas art',
  'dekoration',
  'design',
  'graffiti',
  'graphic design',
  'hamburger',
  'illustration',
  'redskin',
  'stencil'],
 ['4000007', 'calendar', 'date', 'flipper', 'graphic', 'persian'],
 ['4000009',
  'art',
  'branding',
  'design',
  'fesftival',
  'identity',
  'illustration',
  'seed',
  'spring',
  'sprout'],
 ['4000010', 'glitchy', 'titles'],
 ['4000011',
  'budapest',
  'design',
  'graphic design',
  'illustration',
  'tipography',
  't-shirt',
  't-shirt design'],
 ['4000012',
  'blockchain',
  'crypto',
  'cryptocurrency',
  'elix',
  'ethereum',
  'illustration',
  'isometric',
  'lending',
  'mining'],
 ['4000013', 'design', 'news', 'responsive', 'ui', 'ux', 'web', 'website'],
 ['4000015',


In [17]:
count=0
for i in range(0,len(df_temp)):
    line = df_temp[i]
    if len(line) >= 15:
        count+=1
count

346

## 找出所有ui相关的图片

In [3]:
## 从总文件夹中取出符合tag条件的图片 Step2 list 所有id ##
#ui类
ui = []
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "interface", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'the internet', 'web design', 'bitcoin', 'movie', 'computer', 'laptop', "webdesign", "food", "restaurant", "recipe", "facebook", "twitter", "blogging", "dashboard", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]
count = 0
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            count=count+1
            ui.append(df.values[i,0])
            break
#count
print(len(ui))

61703


In [7]:
## 从总文件夹中取出符合tag条件的gif

path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/0 PROJECT/dribbble-final version/UI"

for root, dirs, files in os.walk(path):     
    for i in range(len(ui)):
        a = os.path.isfile(root + '/' + ui[i] + ".gif")
        if a:
            file_path = root+'/' + ui[i] + ".gif"
            new_file_path = new_path+ '/' + ui[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [4]:
# Get all gif ids.
new_path = "D:/0 PROJECT/dribbble-final version/UI"
gif_id = []
for root, dirs, files in os.walk(new_path):
    for i in range(len(files)):
        gif_id.append(files[i][0:7])


In [5]:
tags = df[df.id.isin(gif_id)]
tags

,id,0,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
25,4000044,animation,c4d,design,gif,iphone,minimal,mobile,None,None,...,None,None,None,None,None,None,None,None,None,None
26,4000045,animation,design,homepage,interaction,landing,motion,ui,ux,web,...,None,None,None,None,None,None,None,None,None,None
49,4000084,animation,cool,dribbble,interface,photoshop,planet,None,None,None,...,None,None,None,None,None,None,None,None,None,None
83,4000126,attractive,colorful,creative,design,float,layout,modern,muse,simple,...,None,None,None,None,None,None,None,None,None,None
121,4000188,app,app design,digital publishing,recipe app,ui design,ux design,vegan,None,None,...,None,None,None,None,None,None,None,None,None,None
145,4000220,animation,app,design,gif,hotel,illustrator,photoshop,tourism,ui,...,None,None,None,None,None,None,None,None,None,None
153,4000231,anim,contacts,furniture,industrial,interaction,interface,lamp,motion,ui,...,None,None,None,None,None,None,None,None,None,None
212,4000313,agency,confirmation,dab,form,portfolio,ui,web,website,None,...,None,None,None,None,None,None,None,None,None,None
226,4000338,aep,animation,clothing,design,fashion,loader,logo,reconnect,website,...,None,None,None,None,None,None,None,None,None,None
235,4000353,app,clean,digital,everyday,experience,framer,interface,material,minimal,...,None,None,None,None,None,None,None,None,None,None


In [13]:
# 将“ui”替换为none
ui_tag = ["ui", "user interface","user-interface", "user_interface", "user interface design", "user_interface_design", "uidesign", "ui design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui"]
for i in range(0, len(tags)):
    for j in range(0,tags.shape[1]):
        if tags.values[i,j] in ui_tag:
            tags.values[i,j] = None
tags

,id,0,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
25,4000044,animation,c4d,design,gif,iphone,minimal,mobile,None,None,...,None,None,None,None,None,None,None,None,None,None
26,4000045,animation,design,homepage,interaction,landing,motion,None,ux,web,...,None,None,None,None,None,None,None,None,None,None
49,4000084,animation,cool,dribbble,interface,photoshop,planet,None,None,None,...,None,None,None,None,None,None,None,None,None,None
83,4000126,attractive,colorful,creative,design,float,layout,modern,muse,simple,...,None,None,None,None,None,None,None,None,None,None
121,4000188,app,app design,digital publishing,recipe app,None,ux design,vegan,None,None,...,None,None,None,None,None,None,None,None,None,None
145,4000220,animation,app,design,gif,hotel,illustrator,photoshop,tourism,None,...,None,None,None,None,None,None,None,None,None,None
153,4000231,anim,contacts,furniture,industrial,interaction,interface,lamp,motion,None,...,None,None,None,None,None,None,None,None,None,None
212,4000313,agency,confirmation,dab,form,portfolio,None,web,website,None,...,None,None,None,None,None,None,None,None,None,None
226,4000338,aep,animation,clothing,design,fashion,loader,logo,reconnect,website,...,None,None,None,None,None,None,None,None,None,None
235,4000353,app,clean,digital,everyday,experience,framer,interface,material,minimal,...,None,None,None,None,None,None,None,None,None,None


In [14]:
# delete all none entries in the dataframe.
re_temp = []
for i in range(0, len(tags)):  
    re_temp.append([str(tags.values[i,j]) for j in range(0, tags.shape[1]) if tags.values[i,j]!= None])
    
len(re_temp)

9333

In [6]:
# records_1 is a list of lists, each sublist contains all tags for each design. delete the none entities.
records_1 = []  
for i in range(0, len(tags)):  
    records_1.append([str(tags.values[i,j]) for j in range(1, tags.shape[1]) if tags.values[i,j]!= None])

In [7]:
# find out all tags that has been appeared together with a certain tag.
re = []
ui_tags =  ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "interface", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'the internet', 'web design', 'bitcoin', 'movie', 'computer', 'laptop', "webdesign", "food", "restaurant", "recipe", "facebook", "twitter", "blogging", "dashboard", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]

for line in records_1:
    for i in range(len(line)):
        if line[i] in ui_tags:
            for k in range(len(line)):
                re.append(line[k])
            break
print(len(set(re)))

9340


In [8]:
# counter
from collections import Counter
c = Counter(re)
#dict(c)
c.most_common(1000)

[('ui', 5637),
 ('ux', 3708),
 (' animation', 2613),
 ('design', 1690),
 ('web', 1521),
 ('animation', 1475),
 ('gif', 1461),
 ('motion', 1338),
 ('app', 1322),
 ('mobile', 1243),
 ('interaction', 1210),
 ('website', 1069),
 ('interface', 878),
 ('ios', 801),
 ('illustration', 756),
 (' app', 660),
 ('principle', 655),
 ('minimal', 568),
 ('web design', 480),
 ('iphone', 417),
 ('sketch', 405),
 ('clean', 399),
 ('prototype', 394),
 ('webdesign', 380),
 ('dailyui', 362),
 ('logo', 334),
 ('concept', 314),
 ('icon', 312),
 ('landing', 299),
 ('food', 297),
 ('ui design', 282),
 ('flat', 281),
 ('landing page', 266),
 ('typography', 262),
 (' after effects', 251),
 ('fashion', 244),
 ('transition', 240),
 ('iphone x', 226),
 ('branding', 225),
 ('dashboard', 219),
 ('user interface', 213),
 ('ecommerce', 207),
 ('travel', 206),
 ('music', 205),
 ('portfolio', 201),
 ('uiux', 200),
 (' design', 196),
 ('product', 195),
 ('iphonex', 192),
 ('video', 190),
 ('layout', 187),
 ('vector', 187)

In [33]:
import csv
with open("result.csv", "w", newline = "") as csvfile:
    writer  = csv.writer(csvfile)
    for row in result:
        writer.writerow(row)

In [19]:
tags.to_csv("gif_tags.csv", index=False)

In [ ]:
# delete all none entries in the dataframe.
re_temp = []
for i in range(0, len(re_dataframe)):  
    re_temp.append([str(re_dataframe.values[i,j]) for j in range(0, re_dataframe.shape[1]) if re_dataframe.values[i,j]!= None])
len(re_temp)

In [15]:
# support value = 0.0045, 
from apyori import apriori
association_rules = apriori(re_temp, min_support=0.002, min_confidence=0.05, min_lift=3, min_length=2)
association_results = list(association_rules)  

In [22]:
# 将两个tag以及lift存入csv
result = []
for item in association_results:
    pair = item[0]
    #lift = item[0][2][0][3]
    items = [x for x in pair]
    if len(items) == 2:
        #lift = item[2][0][3]
        #sublist=[items[0],items[1], lift]
        sublist=[items[0],items[1]]
        result.append(sublist)
print(len(result))

238


In [25]:
result_2 = pd.DataFrame(result)
result_2
result_2.to_csv("result.csv", sep=',')

## 没有ui tag 但是是ui类别的图片

In [24]:
# 没有ui tag 但是是ui类别的图片
tag = ["iphone", "iphonex","iphone_x", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template""landing page","app", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp" ]
other = []
#count = 0
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_1:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in tag:
               # count = count+1
                other.append(x)
#len(other)
#count

### 只有website相关tag的图片

In [5]:
web = []
keywords1 = ["iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11","app", "application", "android app", "app-design",  "app design","app_design", "appdesign","ios app", "ios_app", "ios_design", "android_app", "app_development","mobile_application", "application_design", "mobile app","mobileapp","mobile", "mobile design", "mobiledesign", "mobile website", "mobilewebsite","mobile_web"]
keywords2 = ["website","webdesigner", "web-design", "design_for_website","websitedesign", "web app", "mobile_application","mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design"]
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords2:
            web_ = True
            for k in range(0, df.shape[1]):
                if df.values[i,k] in keywords1:
                    web_ = False
                    break
            if web_:
                web.append(df.values[i])
            break
print(len(web))

12533


## 分类取图片

In [4]:
keywords = ["smartphone", "smartphone 2xl", "blog", "homepage",'android', 'iphone x', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'web design', 'movie', 'computer', 'laptop', "webdesign", "facebook", "twitter", "blogging", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]
mobile = ["mobile", "ios","phone","iphone","iphonex","iphone 5", "iphone 6", "iphone 7", "ios11", "smartphone","smartphone 2xl", 'android',"iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","mobile app iphone", "ipad_pro","android_app","mobile_application","mobile app","mobileapp", "mobile", "mobile design"]
map_=["map","mapview"]

In [5]:
## 从总文件夹中取出符合tag条件的图片 Step2 list 所有id ##
# 第一类
class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in map_:
                    class_0.append(df.values[i,0])
                    break

print(len(class_0))

1272


In [21]:
# 第二类
class_1 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] == "ui":
                for k in range(0, df.shape[1]):
                    if df.values[i,k] == "ecommerce" or df.values[i,k] == "e commerce" or df.values[i,k] == "e-commerce" or df.values[i,k] == "commerce":
                        class_1.append(df.values[i, 0])
                        break

print(len(class_1))

660


In [32]:
# 第三类
class_2 = []
a = ["ink",'painting','paintings',"illustration","illustrations", "icon"]
count = 0
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_1 and x not in other:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in a:
                class_2.append(df.values[i,0])
            #if df.values[i,j] == "ui":
                #for k in range(0, df.shape[1]):
                 #   if df.values[i,k] == "green":
                  #      class_2.append(df.values[i, 0])
                   #     break

#print(len(class_2))

In [25]:
# 第四类
class_3 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2:
        for j in range(0, df.shape[1]):
            if df.values[i,j] == "ui":
                for k in range(0, df.shape[1]):
                    if df.values[i,k] == "blue":
                        class_3.append(df.values[i, 0])
                        break

print(len(class_3))

285


In [7]:
## 从总文件夹中取出符合tag条件的图片 Step3 从总文件夹中复制图片到新文件夹 ##

path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Layout/map"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [9]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/App Classification/Images/ecommerce"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [11]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/App Classification/Images/food"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [17]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/App Classification/Images/finance"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_3)):
        a = os.path.isfile(root + '/' + class_3[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_3[i] + ".png")
        if a:
            file_path = root+'/' + class_3[i] + ".jpg"
            new_file_path = new_path+ '/' + class_3[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_3[i] + ".png"
            new_file_path = new_path+ '/' + class_3[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [23]:
## 找出不带ui tag的图片 （数据集中other类） ## 
other = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in ui:
        other.append(df.values[i, 0])

In [11]:
other = pd.DataFrame(other)
other

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,4000002,collection,dunelm,icon,iconography,line,mills,online,set,shop,...,None,None,None,None,None,None,None,None,None,None
1,4000004,canvas,canvas art,dekoration,design,graffiti,graphic design,hamburger,illustration,redskin,...,None,None,None,None,None,None,None,None,None,None
2,4000007,calendar,date,flipper,graphic,persian,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,4000009,art,branding,design,fesftival,identity,illustration,seed,spring,sprout,...,None,None,None,None,None,None,None,None,None,None
4,4000010,glitchy,titles,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,4000011,budapest,design,graphic design,illustration,tipography,t-shirt,t-shirt design,None,None,...,None,None,None,None,None,None,None,None,None,None
6,4000012,blockchain,crypto,cryptocurrency,elix,ethereum,illustration,isometric,lending,mining,...,None,None,None,None,None,None,None,None,None,None
7,4000015,anchor,challenge,design,logo,thirtylogos,thirtylogoschallenge,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,4000016,calligraphy,logotype,rgb blue,single line weight,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,4000018,dj,fever,graffiti,graphic design,illustration,lettering,logo,tag,tipography,...,None,None,None,None,None,None,None,None,None,None


In [16]:
# delete all none entries in the dataframe.
notui = []
for i in range(0, len(other)):  
    notui.append([str(other.values[i,j]) for j in range(0, other.shape[1]) if other.values[i,j]!= None])
    
len(notui)

1000

In [13]:
file = open("notui.txt", 'w')
file.write(str(notui))
file.close()

In [24]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/0 PROJECT/dribbble-final version/UI/notui"

for root, dirs, files in os.walk(path):     
    for i in range(0, 1000):
        a = os.path.isfile(root + '/' + other[i] + ".jpg")
        b = os.path.isfile(root + '/' + other[i] + ".png")
        if a:
            file_path = root+'/' + other[i] + ".jpg"
            new_file_path = new_path+ '/' + other[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + other[i] + ".png"
            new_file_path = new_path+ '/' + other[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [6]:
## 找出不带ui tag的图片 （数据集中other类） ## 
other = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in ui:
        other.append()
other = other[400:700]
        
path = 'D:/01 PROJECT/dribbble-final version/All_images'
new_path = "D:/01 PROJECT/dribbble-final version/UI/test"

for root, dirs, files in os.walk(path):     
    for i in range(0, 300):
        a = os.path.isfile(root + '/' + other[i] + ".jpg")
        b = os.path.isfile(root + '/' + other[i] + ".png")
        if a:
            file_path = root+'/' + other[i] + ".jpg"
            new_file_path = new_path+ '/' + other[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + other[i] + ".png"
            new_file_path = new_path+ '/' + other[i] + ".png"
            shutil.copy(file_path,new_file_path)

### 改图片名


In [18]:
## 改图片名为 类别id_图片id
import os
path_0 = "D:/1 Project Model/App Classification/Images/ecommerce"
for file in os.listdir(path_0):
    os.rename(os.path.join(path_0,file),os.path.join(path_0,"0_"+file))
    
path_1 = "D:/1 Project Model/App Classification/Images/finance"
for file in os.listdir(path_1):
    os.rename(os.path.join(path_1,file),os.path.join(path_1,"1_"+file))
    
path_2 = "D:/1 Project Model/App Classification/Images/food"
for file in os.listdir(path_2):
    os.rename(os.path.join(path_2,file),os.path.join(path_2,"2_"+file))

path_3 = "D:/1 Project Model/App Classification/Images/music"
for file in os.listdir(path_3):
    os.rename(os.path.join(path_3,file),os.path.join(path_3,"3_"+file))
    
path_4 = "D:/1 Project Model/App Classification/Images/travel"
for file in os.listdir(path_4):
    os.rename(os.path.join(path_4,file),os.path.join(path_4,"4_"+file))

In [7]:
path_2 = "D:/1 Project Model/FunctionalityClassification/Images/other_512"
for file in os.listdir(path_2):
    os.rename(os.path.join(path_2,file),os.path.join(path_2,"1_"+file))

### 更改图片尺寸

In [6]:
## 提取目录下所有图片,更改尺寸后保存到另一目录 ##
## training set
path = "D:/1 Project Model/FunctionalityClassification/Images/other"
new_path = "D:/1 Project Model/FunctionalityClassification/Images/other_512"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((512, 512),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [41]:
positive = ['0_4111612.jpg', '0_4005184.png', '0_4110121.png', '0_4008254.png', '0_4004771.jpg', '0_4005200.jpg', '0_4109914.jpg', '0_4008166.jpg', '0_4111145.png', '0_4110346.png', '0_4005420.png', '0_4110356.png', '0_4112007.png', '0_4005983.png', '0_4006893.png', '0_4008454.jpg', '0_4005101.png', '0_4004854.jpg', '0_4114666.png', '0_4005175.png', '0_4112475.jpg', '0_4008437.jpg', '0_4112511.jpg', '0_4110981.png', '0_4110059.png', '0_4009107.jpg', '0_4009110.jpg', '0_4009464.jpg', '0_4004916.png', '0_4008775.jpg', '0_4004505.png', '0_4115254.png', '0_4112478.jpg', '0_4113946.png', '0_4109796.png', '0_4110840.jpg', '0_4004866.jpg', '0_4110430.jpg', '0_4114607.png', '0_4113552.png', '0_4008514.png', '0_4007725.jpg', '0_4008680.png', '0_4111216.png', '0_4115323.jpg', '0_4112693.jpg', '0_4114207.jpg', '0_4006645.jpg', '0_4109755.png', '0_4006103.png', '0_4114732.png', '0_4111321.png', '0_4111394.png', '0_4004712.jpg', '0_4109024.png', '0_4110713.png', '0_4008866.jpg', '0_4110688.jpg', '0_4005238.png', '0_4007584.png', '0_4115010.jpg', '0_4112719.png', '0_4004571.png', '0_4108988.jpg', '0_4112900.jpg', '0_4113258.jpg', '0_4007932.png', '0_4114232.jpg', '0_4009437.jpg', '0_4007430.jpg', '0_4008392.png', '0_4009285.jpg', '0_4111938.jpg', '0_4111727.png', '0_4110132.png', '0_4007808.png', '0_4009354.png', '0_4110909.jpg', '0_4111160.png', '0_4113730.png', '0_4111676.jpg', '0_4006749.png', '0_4005858.jpg', '0_4007199.png']
path = "D:/2 UserStudy/mobile_ecommerce_checkout/result"
new_path = "D:/2 UserStudy/mobile_ecommerce_checkout/result2"

for root, dirs, files in os.walk(path):     
    for i in range(len(positive)):
        file_path = root+'/' + positive[i]
        new_file_path = new_path+ '/' + positive[i]
        shutil.copy(file_path, new_file_path)

In [3]:
## 提取目录下所有图片,更改尺寸后保存到另一目录 ##
## testing set
path = "D:/1 Project Model/Dataset/Func/search&other"
new_path = "D:/1 Project Model/Dataset/Func/search_256"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256,256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

## Color

In [18]:
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "html", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'web design', "webdesign","facebook", "twitter", "blogging", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]

yellow = ["yellow", "orange", "gold"]
red = ["red"]
blue = ["blue", "light blue","dark blue"]
green = ["green", "darkgreen","aquamarine"]
white = ["white"]
black = ["black", "dark"]
pink = ["pink"]
brown = ["brown"]
grey = ["grey"]

In [10]:
# pink
class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in pink:
                    class_0.append(df.values[i,0])
                    break

print(len(class_0))

751


In [19]:
# brown
class_1 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in grey:
                        class_1.append(df.values[i, 0])
                        break
                        
print(len(class_1))

176


In [32]:
# yellow

class_0 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_1 and class_2 and class_3 and class_4 and class_5:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in yellow:
                        class_0.append(df.values[i,0])
                        break

print(len(class_0))

1328


In [11]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Color/pink"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [16]:
# red
class_1 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in red:
                        class_1.append(df.values[i, 0])
                        break
                        
print(len(class_1))

884


In [16]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Color/brown"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [18]:
# green
class_2 = []

for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_1 and x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in green:
                        class_2.append(df.values[i, 0])
                        break

print(len(class_2))

1034


In [19]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Color/green"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [20]:
# blue 类

class_3 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in blue:
                        class_3.append(df.values[i, 0])
                        break

print(len(class_3))

1419


In [21]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Color/blue"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_3)):
        a = os.path.isfile(root + '/' + class_3[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_3[i] + ".png")
        if a:
            file_path = root+'/' + class_3[i] + ".jpg"
            new_file_path = new_path+ '/' + class_3[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_3[i] + ".png"
            new_file_path = new_path+ '/' + class_3[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [27]:
# black 类
class_4 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2 and x not in class_3:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in black:
                        class_4.append(df.values[i, 0])
                        break

print(len(class_4))

1974


In [28]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Color/black"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_4)):
        a = os.path.isfile(root + '/' + class_4[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_4[i] + ".png")
        if a:
            file_path = root+'/' + class_4[i] + ".jpg"
            new_file_path = new_path+ '/' + class_4[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_4[i] + ".png"
            new_file_path = new_path+ '/' + class_4[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [29]:
# white 类
class_5 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2 and x not in class_3 and x not in class_4:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in white:
                        class_5.append(df.values[i, 0])
                        break

print(len(class_5))

1348


In [30]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Color/white"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_5)):
        a = os.path.isfile(root + '/' + class_5[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_5[i] + ".png")
        if a:
            file_path = root+'/' + class_5[i] + ".jpg"
            new_file_path = new_path+ '/' + class_5[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_5[i] + ".png"
            new_file_path = new_path+ '/' + class_5[i] + ".png"
            shutil.copy(file_path,new_file_path)

## Platform

In [3]:
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "html", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'web design', "webdesign","facebook", "twitter", "blogging", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]
ios = ["iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro","ios11",  "ios app", "ios_app", "ios_design", "ios application", "ios_application","ios design", "ios interface","ios ui","ios user interface","mobile","mobile app","mobile app design"]
# web = ['web design', "webdesign","website_mockup", "website_template", "website", "webdesigner", "web-design", "design_for_website","webpage", "web", "website design","webpage design"]

In [6]:
# web相关图片
class_0 = []
keywords1 = ["iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11","app", "application", "android app", "app-design",  "app design","app_design", "appdesign","ios app", "ios_app", "ios_design", "android_app", "app_development","mobile_application", "application_design", "mobile app","mobileapp","mobile", "mobile design", "mobiledesign", "mobile website", "mobilewebsite","mobile_web"]
keywords2 = ["website","webdesigner", "web-design", "design_for_website","websitedesign", "web app", "mobile_application","mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design"]
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords2:
            web_ = True
            for k in range(0, df.shape[1]):
                if df.values[i,k] in keywords1:
                    web_ = False
                    break
            if web_:
                class_0.append(df.values[i, 0])
            break
print(len(class_0))

14256


In [7]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Platform/web"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [8]:
class_1 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in ios:
                        class_1.append(df.values[i, 0])
                        break
                        
print(len(class_1))

23957


In [9]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Platform/ios"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)

## APP type


music = ["music", "music player", "musician", "musical","singer", "apple_music", "punk", "musicplayer","rock_n_roll", "song", "album", "playlist", "hip_hop", "band", "hiphop","music_app", "record", "rapper", "radio", "album_artwork", "jazz", "cd_cover", "rap", "instrument", "piano", "radiohead",  "music_player",  "guitar"]

ecommerce = ["ecommerce", "e commerce", "e-commerce", "commerce", " e_commerce", "e-shop", "conversion", "items", "check_out", "online_store", "digital_reading", "ikea", "eshop", "online_shop", "woocommerce", "cart", "commerce", "digital_bookstore", "commercial", "adidas", "nike","footwear", "clothing", "clothes", "fashion", "cosmetic","beauty_salon","electronics", "beauty","accessories", "webshop"]

food = ["hamburger", "food","foods", "dish", "fastfood", "fast_food", "food_blog", "menu","cooking","cook", "cheeseburger", "burger", "burgers", "meal","eat", "noodle", "food_menu", "dessert", "juice",  "drink", "pizza", "restaurant","foodie",  "kitchen", "chili", "junk_food", "food_truck", "lunch", "yogurt","food_illustration", "bakery", "bake","baker", "fries","bread","cake", "cupcake", "caffe", "cuisine","recipe", "recipes", "restaurants", "food_menu", "food_app", "restaurant_branding", "sushi", "drinks", "cocktail", "alcohol","breakfast","dinner","cocktails","vegetables","pasta","seafood","cookies","waffle","sandwich", "hot_chocolate","hot_dog", "pancakes", "dish", "beverage","martini","brunch","juice","vegetarian", "soup","veggie","rice","strawberry","steak","smoothie","salad","veggies","peanut","mexican","pepperoni"]

travel = ["travel", "travel_guide","airbnb","flights","hiking","travelling","national_parks","national_park","travel_agency","traveler","tourist","travel_app","travelogue","travelling","tours","tourism","roadtrip","trip","vacation","travelapp","travel app","travelapps","travel application","travel applications","travel_applications","flight_booking"]

finance = ["bank", "cash","crypto_currency","budget","bitcoin","finance","btc","loan","banking","finances","finance_app","financial","investing","wallet","invest","investment","crypto","cryptocurrency","litecoin","currency","blockchain","crypto_currency","cryptocurrencies","small_business","business_app","insurance","marketing"]

In [5]:
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "interface", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'the internet', 'web design', "webdesign","facebook", "twitter", "blogging", "dashboard", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]
music = ["music", "music player", "musicplayer", "music_player","music-player","music players","musicplayers","music_players","apple_music", "apple music","apple-music","playlist", "music app","musicapp","music_app", "music application","music_applications", "music applications","music player app","music player ui"]
ecommerce = ["ecommerce","e commerce", "e-commerce", "commerce", " e_commerce", "e-shop","online_store","ikea", "eshop", "online_shop","adidas", "nike","footwear", "clothing", "clothes","webshop"]
food = ["salad","bakery", "steak", "cake","recipe","recipes","food","foods", "dessert", "juice","drink", "food_app","food app","food application", "foodapp","food_application","restaurant"]
travel = ["travel", "hiking","travelling","national_parks","national_park","travel_agency","tourist","travel_app", "tours","tourism","roadtrip","trip","vacation","travelapp","travel app","travelapps","travel application","travel applications","travel_applications"]
finance = ["bank", "finance", "banking","finances","finance_app","finance app","finance application","finance_app","finance_application","banking app","banding_app","bank application","bank app","bank_app","financial","investing","insurance","wallet"]

In [9]:
game = ["game","videogame","video game","games"]
health = ["health", "healthy","fitness"]
news = ["news","newspaper","news design",  "news grid",  "news list",  "newstemplate", "news app","news feed"]
sport = ["sport","sports","gym","workout","exercising","exercise","exercises"]
social=["socialnetwork", "social network", "social networking", "blog", "messenger", "facebook", 'instagram', 'dating', 'chat',"chatting"]
weather = ["weather","weather app","weather_app","temperature"]
#lifestyle = ["fashion", "furniture", "real estate", "real_estate"]
medical = [ "medical", "healthcare", "hospital", "pharmacy", "medicine"]
book = ["magazine", "magazines", "reading", "bookstore", "digitalreading", "digital reading", "digital_reading", "digital bookstroe", "digital_bookstroe","digitalbookstroe", "book","books"]


In [22]:

# music 类
class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in book:
                    class_0.append(df.values[i,0])
                    break

print(len(class_0))

899


In [23]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/App Classification/Images/book"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [26]:
# food & drink 类
# 第二类
class_1 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_2 and x not in class_3 and x not in class_4:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in food:
                        class_1.append(df.values[i, 0])
                        break

print(len(class_1))

2338


In [27]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/App Classification/Images/food"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [10]:
# travel类 
# 第三类
class_2 = []

for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_1 and x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in travel:
                        class_2.append(df.values[i, 0])
                        break

print(len(class_2))

2751


In [11]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/App Classification/Images/travel"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [12]:
# finance 类

# 第四类
class_3 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in finance:
                        class_3.append(df.values[i, 0])
                        break

print(len(class_3))

2364


In [13]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/App Classification/Images/finance"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_3)):
        a = os.path.isfile(root + '/' + class_3[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_3[i] + ".png")
        if a:
            file_path = root+'/' + class_3[i] + ".jpg"
            new_file_path = new_path+ '/' + class_3[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_3[i] + ".png"
            new_file_path = new_path+ '/' + class_3[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [14]:
# ecommerce 类
# 第五类
class_4 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2 and x not in class_3:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in ecommerce:
                        class_4.append(df.values[i, 0])
                        break

print(len(class_4))

5863


In [15]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/App Classification/Images/ecommerce"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_4)):
        a = os.path.isfile(root + '/' + class_4[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_4[i] + ".png")
        if a:
            file_path = root+'/' + class_4[i] + ".jpg"
            new_file_path = new_path+ '/' + class_4[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_4[i] + ".png"
            new_file_path = new_path+ '/' + class_4[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [7]:
## 提取目录下所有图片,更改尺寸后保存到另一目录 ##
path = "D:/1 Project Model/Dataset/Layout/chart"
new_path = "D:/1 Project Model/Dataset/Layout/4"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [25]:
## 提取目录下所有图片,更改尺寸后保存到另一目录 ##
path = "D:/1 Project Model/App Classification/Images/book"
new_path = "D:/1 Project Model/App Classification/Images/book_256"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [30]:
## 提取目录下所有图片,更改尺寸后保存到另一目录 ##
path = "D:/1 Project Model/App Classification/Images/travel"
new_path = "D:/1 Project Model/App Classification/Images/2"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [12]:
## 提取目录下所有图片,更改尺寸后保存到另一目录 ##
path = "D:/1 Project Model/Dataset/Color/pink"
new_path = "D:/1 Project Model/Dataset/Color/6"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [35]:
## 提取目录下所有图片,更改尺寸后保存到另一目录 ##
path = "D:/1 Project Model/App Classification/Images/news"
new_path = "D:/1 Project Model/App Classification/Images/news_256"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

C:\Users\DELL\Anaconda3\lib\site-packages\PIL\Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [36]:
import os
path_0 = "D:/1 Project Model/App Classification/Images/news_256"
for file in os.listdir(path_0):
    os.rename(os.path.join(path_0,file),os.path.join(path_0,"0_"+file))

## Functionality

In [4]:
# landing page 类
class_0 = []
landing = ["landing page", "landing pages","landingpage","landingpages","landing_pages","landing_page"]
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in health:
                    class_0.append(df.values[i,0])
                    break

print(len(class_0))

NameError: name 'keywords' is not defined

In [5]:
# check out 类
# 第二类
checkout = [ "check_out", "check out", "checkout", "payment"]
class_1 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in checkout:
                        class_1.append(df.values[i, 0])
                        break

print(len(class_1))

1395


In [8]:
# sign up 类 
# 第三类
class_2 = []
signup = ["sign up","signup","sign_up","login","log in","log_in"]
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_1 and x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in signup:
                        class_2.append(df.values[i, 0])
                        break

print(len(class_2))

3496


In [7]:
# profile类
# 第四类
class_3 = []
profile = ["profile"]
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in profile:
                        class_3.append(df.values[i, 0])
                        break

print(len(class_3))

2844


In [41]:
# search

class_4 = []
search = ["search", "searching"]
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in search:
                    class_4.append(df.values[i,0])
                    break

print(len(class_4))

1087


In [42]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Func/search"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_4)):
        a = os.path.isfile(root + '/' + class_4[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_4[i] + ".png")
        if a:
            file_path = root+'/' + class_4[i] + ".jpg"
            new_file_path = new_path+ '/' + class_4[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_4[i] + ".png"
            new_file_path = new_path+ '/' + class_4[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [22]:
## 从总文件夹中取出符合tag条件的图片 Step3 从总文件夹中复制图片到新文件夹 ##

path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Func/game"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [10]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/FunctionalityClassification/Images/checkout"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [11]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/FunctionalityClassification/Images/signup"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [12]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/FunctionalityClassification/Images/profile"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_3)):
        a = os.path.isfile(root + '/' + class_3[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_3[i] + ".png")
        if a:
            file_path = root+'/' + class_3[i] + ".jpg"
            new_file_path = new_path+ '/' + class_3[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_3[i] + ".png"
            new_file_path = new_path+ '/' + class_3[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [13]:
import os
path_0 = "D:/1 Project Model/FunctionalityClassification/Images/landing"
for file in os.listdir(path_0):
    os.rename(os.path.join(path_0,file),os.path.join(path_0,"0_"+file))
    
path_1 = "D:/1 Project Model/FunctionalityClassification/Images/checkout"
for file in os.listdir(path_1):
    os.rename(os.path.join(path_1,file),os.path.join(path_1,"1_"+file))
    
path_2 = "D:/1 Project Model/FunctionalityClassification/Images/signup"
for file in os.listdir(path_2):
    os.rename(os.path.join(path_2,file),os.path.join(path_2,"2_"+file))

path_3 = "D:/1 Project Model/FunctionalityClassification/Images/profile"
for file in os.listdir(path_3):
    os.rename(os.path.join(path_3,file),os.path.join(path_3,"3_"+file))

In [23]:
## 提取目录下所有图片,更改尺寸后保存到另一目录 ##
## training set
path = "D:/1 Project Model/Dataset/Func/game"
new_path = "D:/1 Project Model/Dataset/Func/game&other"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [15]:
path = "D:/1 Project Model/FunctionalityClassification/Images/checkout"
new_path = "D:/1 Project Model/FunctionalityClassification/Images/1"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [16]:
path = "D:/1 Project Model/FunctionalityClassification/Images/signup"
new_path = "D:/1 Project Model/FunctionalityClassification/Images/2"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [17]:
path = "D:/1 Project Model/FunctionalityClassification/Images/profile"
new_path = "D:/1 Project Model/FunctionalityClassification/Images/3"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [25]:
# search
path = "D:/1 Project Model/Dataset/Func/search/test/search"
new_path = "D:/1 Project Model/Dataset/Func/search/test/0"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [26]:
import os
path_0 = "D:/1 Project Model/Dataset/Func/game&other"
for file in os.listdir(path_0):
    os.rename(os.path.join(path_0,file),os.path.join(path_0,"0_"+file))

## Layout


In [82]:
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "interface", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'the internet', 'web design', "webdesign","facebook", "twitter", "blogging", "dashboard", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]
dashboard = ["dashboard"]
list_ = ["list"]
form = ["form"]
grid = ["grid"]
chart = ["chart"]

In [86]:

# grid 类
class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in grid:
                    class_0.append(df.values[i,0])
                    break

print(len(class_0))

1109


In [87]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Layout/grid"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [89]:
# chart类
# 第二类
class_1 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in chart:
                        class_1.append(df.values[i, 0])
                        break

print(len(class_1))

1005


In [92]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Layout/chart"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [93]:
# form类 
# 第三类
class_2 = []

for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_1 and x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in form:
                        class_2.append(df.values[i, 0])
                        break

print(len(class_2))

1103


In [94]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Layout/form"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [95]:
# list类

# 第四类
class_3 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in list_:
                        class_3.append(df.values[i, 0])
                        break

print(len(class_3))

739


In [96]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Layout/list"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_3)):
        a = os.path.isfile(root + '/' + class_3[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_3[i] + ".png")
        if a:
            file_path = root+'/' + class_3[i] + ".jpg"
            new_file_path = new_path+ '/' + class_3[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_3[i] + ".png"
            new_file_path = new_path+ '/' + class_3[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [97]:
# dashboard类
# 第五类
class_4 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2 and x not in class_3:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in dashboard:
                        class_4.append(df.values[i, 0])
                        break

print(len(class_4))

4830


In [98]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Layout/dashboard"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_4)):
        a = os.path.isfile(root + '/' + class_4[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_4[i] + ".png")
        if a:
            file_path = root+'/' + class_4[i] + ".jpg"
            new_file_path = new_path+ '/' + class_4[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_4[i] + ".png"
            new_file_path = new_path+ '/' + class_4[i] + ".png"
            shutil.copy(file_path,new_file_path)

## Style

In [105]:
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "interface", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'the internet', 'web design', "webdesign","facebook", "twitter", "blogging", "dashboard", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]
simple = ["simple","clean", "minimal","minimalistic"]
flat = ["flat","flat design","flat_design","flatdesign","flat ui","flat_ui_design"]
gradient = ["color gradient", "color_gradient", "gradient color","gradient"]
illustration = ["illustration"]


In [102]:

# gradient 类
class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in gradient:
                    class_0.append(df.values[i,0])
                    break

print(len(class_0))

3121


In [104]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Style/gradient"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [106]:
# flat类
# 第二类
class_1 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in flat:
                        class_1.append(df.values[i, 0])
                        break

print(len(class_1))

4915


In [107]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Style/flat"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [108]:
# illustration类 
# 第三类
class_2 = []

for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_1 and x not in class_0:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in illustration:
                        class_2.append(df.values[i, 0])
                        break

print(len(class_2))

6808


In [109]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Style/illustration"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [110]:
# simple类

# 第四类
class_3 = []
for i in range(0, len(df)):
    x = df["id"][i]
    if x not in class_0 and x not in class_1 and x not in class_2:
        for j in range(0, df.shape[1]):
            if df.values[i,j] in keywords:
                for k in range(0, df.shape[1]):
                    if df.values[i,k] in simple:
                        class_3.append(df.values[i, 0])
                        break

print(len(class_3))

12286


In [111]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/1 Project Model/Dataset/Style/simple"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_3)):
        a = os.path.isfile(root + '/' + class_3[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_3[i] + ".png")
        if a:
            file_path = root+'/' + class_3[i] + ".jpg"
            new_file_path = new_path+ '/' + class_3[i] + ".jpg"
            shutil.copy(file_path,new_file_path)
        if b:
            file_path = root+'/' + class_3[i] + ".png"
            new_file_path = new_path+ '/' + class_3[i] + ".png"
            shutil.copy(file_path,new_file_path)

# USER STUDY

## CASE 1: mobile+ecommerce+checkout

In [ ]:
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'web design', 'movie', 'computer', 'laptop', "webdesign", "facebook", "twitter", "blogging", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]
mobile = ["mobile", "ios","phone","iphone","iphonex","iphone 5", "iphone 6", "iphone 7", "ios11", "smartphone","smartphone 2xl", 'android',"iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","mobile app iphone", "ipad_pro","android_app","mobile_application","mobile app","mobileapp", "mobile", "mobile design"]


In [3]:
## 从总文件夹中取出符合tag条件的图片 Step2 list 所有id ##
#ui类
ui = []
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'web design', 'movie', 'computer', 'laptop', "webdesign", "facebook", "twitter", "blogging", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]
count = 0
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            count=count+1
            ui.append(df.values[i,0])
            break
#count
print(len(ui))

57831


In [37]:
# case 1: mobile + ecommerce + checkout
class_0 = []
iphone = ["ios","iphonex"]
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] == "checkout":
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] == "ecommerce":
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] in iphone:
                                    class_0.append(df.values[i,0])
                                    break

print(len(class_0))
print(len(set(class_0)))

9
3


In [17]:
class_1 = []
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'web design', 'movie', 'computer', 'laptop', "webdesign", "facebook", "twitter", "blogging", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]

case1 = ["mobile", "ecommerce", "checkout"]
mobile = ["mobile", "ios","phone","iphone","iphonex","iphone 5", "iphone 6", "iphone 7", "ios11", "smartphone","smartphone 2xl", 'android',"iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","mobile app iphone", "ipad_pro","android_app","mobile_application","mobile app","mobileapp", "mobile", "mobile design"]
ecommerce = ["ecommerce","e commerce", "e-commerce", "commerce", "online shop","online store","shopping", "e_commerce", "e-shop","online_store","ikea", "eshop", "ecommerce shop", "e-commerce store", "online_shop","adidas", "nike","footwear", "clothing", "clothes","webshop"]
checkout = ["checkout", "check out", "check_out", "credit card payment", "payment", "payments", "credit card", "checkout page", "checkout purchase buy add item", "pay", "checkout pay card","purchase", "checkout.","card checkout", "creditcard", "credit card form", "payment form", "credit", "mastercard"]
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in checkout:
                    for x in range(0, df.shape[1]):
                        #if df.values[i, x] in ecommerce:
                            #for y in range(0, df.shape[1]):
                                if df.values[i, x] in mobile:
                                    class_1.append(df.values[i,0])
                                    break

print(len(class_1))

941


In [38]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case1"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        c = os.path.isfile(root + '/' + class_0[i] + ".gif")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_0[i] + ".gif"
            new_file_path = new_path+ '/' + class_0[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [26]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case1/mobile+ecommerce"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        #c = os.path.isfile(root + '/' + class_1[i] + ".gif")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)
        #if c:
         #   file_path = root+'/' + class_1[i] + ".gif"
          #  new_file_path = new_path+ '/' + class_1[i] + ".gif"
           # shutil.copy(file_path,new_file_path)

 将mobile的图片取出来，看能加ecommerce和checkout tag的图片有多少

In [32]:
class_2 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in mobile:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in ecommerce:
                            class_2.append(df.values[i,0])
                            break
print(len(class_2))

2017


In [33]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case1/x"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [21]:
path = "D:/2 UserStudy/Case1/ecommerce&checkout"
new_path = "D:/2 UserStudy/Case1/mobile_test"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [22]:
import os
path_0 = "D:/2 UserStudy/Case1/mobile_test"
for file in os.listdir(path_0):
    os.rename(os.path.join(path_0,file),os.path.join(path_0, "0_"+file))

In [40]:
# 获取所有的id
path = "D:/2 UserStudy/Case1/result"
original_path = "D:/2 UserStudy/Case1/x"
new_path = "D:/2 UserStudy/Case1/y"
fnames = []
for file in os.listdir(path):
    fname = file[2:]
    fnames.append(fname)
    
for root, dirs, files in os.walk(original_path):
    for i in range(len(fnames)):
        file_path = root+'/'+fnames[i]
        new_file_path = new_path+"/"+fnames[i]
        shutil.copy(file_path, new_file_path)

In [26]:
positive = ['0_4178273.png', '0_4177558.jpg', '0_4177849.jpg', '0_4178033.jpg', '0_4178650.png', '0_4177203.png', '0_4177470.png', '0_4175353.jpg', '0_4178939.jpg', '0_4175248.jpg', '0_4178074.jpg', '0_4176699.png', '0_4178406.jpg', '0_4175313.jpg', '0_4178361.jpg', '0_4175383.png', '0_4178291.jpg', '0_4175904.jpg', '0_4177935.png', '0_4177622.jpg', '0_4175984.png', '0_4177442.png', '0_4178639.jpg', '0_4178862.jpg', '0_4178390.jpg', '0_4177186.jpg', '0_4176235.jpg', '0_4176009.jpg', '0_4178325.jpg', '0_4176311.jpg', '0_4178713.jpg', '0_4177007.jpg']

path = "D:/2 UserStudy/web_signup_form/result"
new_path = "D:/2 UserStudy/web_signup_form/result2"

for root, dirs, files in os.walk(path):     
    for i in range(len(positive)):
        file_path = root+'/' + positive[i]
        new_file_path = new_path+ '/' + positive[i]
        shutil.copy(file_path, new_file_path)

## CASE 2: mobile + food + list

In [7]:
# case 2: mobile + food + list
class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] == "food":
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] == "list":
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] == "iphone":
                                    class_0.append(df.values[i,0])
                                    break

print(len(class_0))

2


In [34]:
mobile = ["mobile", "ios","phone","iphone","iphonex","iphone 5", "iphone 6", "iphone 7", "ios11", "smartphone","smartphone 2xl", 'android',"iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro","android_app","mobile_application","mobile app","mobileapp", "mobile", "mobile design"]
food = ["salad","bakery", "steak", "cake","recipe","recipes","food","foods", "dessert", "juice","drink", "food_app","food app","food application", "foodapp","food_application","restaurant"]
list_ = ["list", "listing", "product list","product listing","product_list","product_listing", "menu", "listing design"]
other=["list ui", "product page"]
#signup = ["sign up","signup","sign_up","login","log in","log_in", "signin", "sign in", "sign_in", "logup", "log up", "log_up"]

In [24]:
class_1 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in mobile:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in food:
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] in list_:
                                    class_1.append(df.values[i,0])
                                    break

print(len(class_1))
print(len(set(class_1)))

97
23


In [8]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case2"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        c = os.path.isfile(root + '/' + class_0[i] + ".gif")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_0[i] + ".gif"
            new_file_path = new_path+ '/' + class_0[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [25]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case2/more search"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        c = os.path.isfile(root + '/' + class_1[i] + ".gif")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_1[i] + ".gif"
            new_file_path = new_path+ '/' + class_1[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [35]:
class_2 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in other:
                    #for x in range(0, df.shape[1]):
                        #if df.values[i, x] in mobile:
                            class_2.append(df.values[i,0])
                            break
print(len(class_2))
print(len(set(class_2)))

360
134


In [36]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case2/other"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [15]:
path = "D:/2 UserStudy/Case2/x"
new_path = "D:/2 UserStudy/Case2/signin_test"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [16]:
import os
path_0 = "D:/2 UserStudy/Case2/signin_test"
for file in os.listdir(path_0):
    os.rename(os.path.join(path_0,file),os.path.join(path_0,"0_"+file))

In [25]:
# 将符合条件的图片从所有测试集中取出，放进result
positive = ['0_4099648.jpg', '0_4045822.png', '0_4269369.png', '0_4026113.jpg', '0_4091824.png', '0_4085459.jpg', '0_4012118.png', '0_4197492.jpg', '0_4062448.jpg', '0_4103239.jpg', '0_4067520.png', '0_4023476.png', '0_4176860.png', '0_4171701.jpg', '0_4036348.png', '0_4188446.png', '0_4025218.jpg', '0_4023165.png', '0_4125340.png', '0_4115735.png', '0_4148408.jpg', '0_4275513.png', '0_4110472.jpg', '0_4044681.png', '0_4148212.png', '0_4271001.png', '0_4240253.png', '0_4011394.png', '0_4172638.png', '0_4050551.png', '0_4255180.jpg', '0_4047999.jpg', '0_4027156.jpg', '0_4016985.png', '0_4245516.png', '0_4189213.png', '0_4276125.png', '0_4290986.jpg', '0_4201829.png', '0_4033699.jpg', '0_4033588.png', '0_4138017.png', '0_4101825.png', '0_4291765.png', '0_4009110.jpg', '0_4118475.png', '0_4019893.png', '0_4008775.jpg', '0_4118581.png', '0_4261868.png', '0_4210339.png', '0_4250023.png', '0_4026826.jpg', '0_4103078.png', '0_4325404.png', '0_4074627.png', '0_4282423.jpg', '0_4032897.png', '0_4136295.jpg', '0_4059024.jpg', '0_4316855.jpg', '0_4022806.jpg', '0_4039707.jpg', '0_4263781.png', '0_4240903.png', '0_4044648.png', '0_4264392.png', '0_4279084.jpg', '0_4265562.png', '0_4047989.png', '0_4095434.png', '0_4164920.png', '0_4042187.jpg', '0_4129384.jpg', '0_4302589.png', '0_4322943.png', '0_4037444.jpg', '0_4118904.png', '0_4055628.jpg', '0_4192008.jpg', '0_4150722.png', '0_4111537.jpg', '0_4025089.png', '0_4095828.jpg', '0_4014379.jpg', '0_4276880.png', '0_4138090.png', '0_4168280.jpg', '0_4192283.png', '0_4002687.jpg', '0_4010606.png', '0_4089208.png', '0_4160996.png', '0_4027968.jpg', '0_4167444.jpg', '0_4033706.jpg', '0_4026744.jpg', '0_4193037.png', '0_4064948.jpg', '0_4198621.png', '0_4236615.png', '0_4328936.jpg', '0_4026155.png', '0_4016408.jpg', '0_4152435.jpg', '0_4087765.png', '0_4026123.jpg', '0_4049553.jpg', '0_4038878.png', '0_4170235.jpg', '0_4075473.jpg', '0_4235379.jpg', '0_4240961.png', '0_4207717.jpg', '0_4152580.jpg', '0_4053988.jpg', '0_4307496.jpg', '0_4326134.png', '0_4259670.jpg', '0_4201054.png', '0_4034070.png', '0_4259181.png', '0_4107997.jpg', '0_4196643.png', '0_4068952.png', '0_4087715.jpg', '0_4327722.jpg', '0_4278769.jpg', '0_4007388.jpg', '0_4218941.png', '0_4286563.jpg', '0_4133070.png', '0_4079067.png', '0_4260344.jpg', '0_4293362.png', '0_4293100.jpg', '0_4017126.jpg', '0_4068852.jpg', '0_4301900.jpg', '0_4281549.jpg', '0_4085207.jpg', '0_4177511.jpg', '0_4114868.jpg', '0_4189649.png', '0_4242522.jpg', '0_4160107.png', '0_4225221.png', '0_4042616.jpg', '0_4203702.jpg', '0_4166361.jpg', '0_4106987.jpg', '0_4188451.jpg', '0_4234561.png', '0_4072759.jpg', '0_4056662.jpg', '0_4276172.png', '0_4046142.jpg', '0_4213065.png', '0_4251896.png', '0_4291569.png', '0_4085412.png', '0_4196190.jpg', '0_4139326.jpg', '0_4026121.jpg', '0_4245372.jpg', '0_4026114.jpg', '0_4172390.png', '0_4260829.jpg', '0_4082638.png', '0_4032150.jpg', '0_4032572.png', '0_4051708.png', '0_4268091.png', '0_4046738.jpg', '0_4202475.png', '0_4013966.png', '0_4288535.jpg', '0_4229629.png', '0_4049332.png', '0_4101370.png', '0_4203073.png', '0_4116066.png']

path = "D:/2 UserStudy/Case2/list_test"
new_path = "D:/2 UserStudy/Case2/result"

for root, dirs, files in os.walk(path):     
    for i in range(len(positive)):
        file_path = root+'/' + positive[i]
        new_file_path = new_path+ '/' + positive[i]
        shutil.copy(file_path, new_file_path)

In [26]:
# 获取所有的id
path = "D:/2 UserStudy/Case2/result"
original_path = "D:/2 UserStudy/Case2/x"
new_path = "D:/2 UserStudy/Case2/y"
fnames = []
for file in os.listdir(path):
    fname = file[2:]
    fnames.append(fname)
    
for root, dirs, files in os.walk(original_path):
    for i in range(len(fnames)):
        file_path = root+'/'+fnames[i]
        new_file_path = new_path+"/"+fnames[i]
        shutil.copy(file_path, new_file_path)

## CASE 3: website + landing page + travel

In [27]:
# case 3: website + landing page + travel

class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] == "travel":
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] == "landing page":
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] == "website":
                                    class_0.append(df.values[i,0])
                                    break

print(len(class_0))

112


In [28]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case3/Base"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        c = os.path.isfile(root + '/' + class_0[i] + ".gif")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_0[i] + ".gif"
            new_file_path = new_path+ '/' + class_0[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [18]:
web = ["website","webdesigner", "web-design", "design_for_website","websitedesign", "web app","webpage", "website", "web", "website design","webpage design"]
travel = ["travel", "hiking","travelling","national_parks","national_park","travel_agency","tourist","travel_app", "tours","tourism","roadtrip","trip","vacation","travelapp","travel app","travelapps","travel application","travel applications","travel_applications"]
landing = ["landing page", "landing pages","landingpage","landingpages","landing_pages","landing_page"]

In [30]:
class_1 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in web:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in landing:
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] in travel:
                                    class_1.append(df.values[i,0])
                                    break

print(len(class_1))

268


In [31]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case3/Enhanced"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        c = os.path.isfile(root + '/' + class_1[i] + ".gif")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_1[i] + ".gif"
            new_file_path = new_path+ '/' + class_1[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [21]:
class_2 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in landing:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in web:
                            class_2.append(df.values[i,0])
                            break
print(len(class_2))

6417


In [22]:
print(len(set(class_2)))

1694


In [36]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case3/x"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [37]:
path = "D:/2 UserStudy/Case3/x"
new_path = "D:/2 UserStudy/Case3/landing_test"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [38]:
import os
path_0 = "D:/2 UserStudy/Case3/landing_test"
for file in os.listdir(path_0):
    os.rename(os.path.join(path_0,file),os.path.join(path_0,"0_"+file))

In [39]:
positive = ['0_4031204.png', '0_4097076.png', '0_4272008.jpg', '0_4226231.jpg', '0_4215497.jpg', '0_4003527.png', '0_4237598.jpg', '0_4130513.jpg', '0_4281457.png', '0_4318129.jpg', '0_4292298.png', '0_4231343.jpg', '0_4213186.png', '0_4057635.png', '0_4302581.jpg', '0_4179644.jpg', '0_4108017.png', '0_4174192.png', '0_4232009.jpg', '0_4186381.jpg', '0_4105488.png', '0_4105609.png', '0_4165888.jpg', '0_4300509.jpg', '0_4004124.png', '0_4143832.jpg', '0_4267424.jpg', '0_4287594.png', '0_4097611.jpg', '0_4100279.png', '0_4042704.png', '0_4067810.png', '0_4011204.png', '0_4016551.png', '0_4213199.png', '0_4281126.png', '0_4097061.jpg', '0_4245251.png', '0_4018261.jpg', '0_4162224.png', '0_4185674.jpg', '0_4205439.jpg', '0_4083015.jpg', '0_4168932.jpg', '0_4003430.png', '0_4133753.jpg', '0_4017992.jpg', '0_4217916.jpg', '0_4054383.jpg', '0_4023948.jpg', '0_4252742.png', '0_4036434.jpg', '0_4110524.png', '0_4200357.png', '0_4061855.png', '0_4050470.jpg', '0_4262700.png', '0_4212711.jpg', '0_4071495.jpg', '0_4028183.jpg', '0_4325284.png', '0_4043457.png', '0_4141393.jpg', '0_4139975.png', '0_4220826.jpg', '0_4218079.jpg', '0_4258907.jpg', '0_4273816.jpg', '0_4038470.jpg', '0_4165011.jpg', '0_4016739.png', '0_4065987.jpg', '0_4213194.png', '0_4105109.png', '0_4060303.png', '0_4081449.jpg', '0_4117792.png', '0_4191404.jpg', '0_4276291.png', '0_4304043.png', '0_4303551.png', '0_4145875.png', '0_4244117.jpg', '0_4213434.jpg', '0_4234086.jpg', '0_4217932.png', '0_4260999.png', '0_4183423.jpg', '0_4230104.jpg', '0_4215236.jpg', '0_4167944.png', '0_4208076.jpg', '0_4272186.png', '0_4232093.jpg', '0_4019954.png', '0_4058392.jpg', '0_4234054.png', '0_4199742.png', '0_4102751.png', '0_4238316.png', '0_4130763.jpg', '0_4107240.jpg', '0_4221740.png', '0_4227085.png', '0_4282165.jpg', '0_4226931.png', '0_4187946.png', '0_4007859.png', '0_4221674.jpg', '0_4137752.jpg', '0_4178470.png', '0_4279645.png', '0_4147845.jpg', '0_4210962.png', '0_4319184.png', '0_4250775.jpg', '0_4178375.jpg', '0_4151723.jpg', '0_4312728.png', '0_4037698.jpg', '0_4102453.png', '0_4080652.jpg', '0_4283059.png', '0_4023909.png', '0_4291413.png', '0_4164920.png', '0_4326229.png', '0_4177877.png', '0_4306627.jpg', '0_4012548.png', '0_4011916.jpg', '0_4060671.jpg', '0_4028002.jpg', '0_4184494.png', '0_4136803.jpg', '0_4165427.jpg', '0_4011165.png', '0_4072115.png', '0_4017217.jpg', '0_4184544.jpg', '0_4252907.jpg', '0_4180998.jpg', '0_4164739.jpg', '0_4058393.jpg', '0_4122099.png', '0_4238383.png', '0_4017737.jpg', '0_4216201.jpg', '0_4268381.jpg', '0_4186785.png', '0_4314734.png', '0_4198382.jpg', '0_4236539.jpg', '0_4283763.png', '0_4210595.png', '0_4155615.png', '0_4039335.jpg', '0_4216885.png', '0_4029809.png', '0_4215614.png', '0_4181549.png', '0_4094750.jpg', '0_4209874.png', '0_4149101.png', '0_4019599.png', '0_4001156.png', '0_4315419.png', '0_4083193.jpg', '0_4242414.png', '0_4230280.jpg', '0_4250480.jpg', '0_4286971.png', '0_4030446.png', '0_4178104.png', '0_4114999.png', '0_4322389.jpg', '0_4213828.png', '0_4160031.png', '0_4315722.png', '0_4313964.jpg', '0_4059608.png', '0_4259372.jpg', '0_4149085.png', '0_4270351.png', '0_4097053.png', '0_4314623.png', '0_4103465.jpg', '0_4182464.png', '0_4272087.jpg', '0_4249770.jpg', '0_4105504.jpg', '0_4053704.png', '0_4268605.png', '0_4083673.png', '0_4036144.jpg', '0_4106690.png', '0_4312150.jpg', '0_4042256.png', '0_4298227.png', '0_4213195.png', '0_4147927.png', '0_4144163.jpg', '0_4291660.jpg', '0_4318388.png', '0_4221982.png', '0_4175827.png', '0_4258113.png', '0_4009224.png', '0_4082357.jpg', '0_4200386.jpg', '0_4259912.png', '0_4084589.jpg', '0_4044261.jpg', '0_4213256.jpg', '0_4048386.jpg', '0_4143842.jpg', '0_4215530.jpg', '0_4011619.png', '0_4197632.png', '0_4235191.png', '0_4034023.jpg', '0_4204144.png', '0_4200865.png', '0_4218948.png', '0_4246112.png', '0_4034841.jpg', '0_4220487.jpg', '0_4096802.jpg', '0_4301845.jpg', '0_4036887.png', '0_4244698.jpg', '0_4125403.jpg', '0_4228247.jpg', '0_4136174.png', '0_4247371.png', '0_4218977.png', '0_4243953.jpg', '0_4258697.jpg', '0_4108236.png', '0_4215977.jpg', '0_4246350.png', '0_4115147.png', '0_4062921.png', '0_4135283.jpg', '0_4046746.png', '0_4222278.jpg', '0_4151427.png', '0_4251377.png', '0_4186659.png', '0_4057353.png', '0_4233481.png', '0_4242596.png', '0_4182276.png', '0_4126848.png', '0_4202477.png', '0_4238058.png', '0_4010898.jpg', '0_4184225.png', '0_4313212.png', '0_4076610.jpg', '0_4213658.png', '0_4202775.jpg', '0_4059625.png', '0_4097007.jpg', '0_4238121.png', '0_4239043.jpg', '0_4250855.png', '0_4131519.png', '0_4028390.png', '0_4015744.png', '0_4060514.jpg', '0_4110984.png', '0_4199845.png', '0_4149108.png', '0_4187259.png', '0_4020613.png', '0_4205320.png', '0_4243124.jpg', '0_4220079.png', '0_4302121.png', '0_4051229.png', '0_4242789.png', '0_4213730.png', '0_4234790.png', '0_4247566.jpg', '0_4293585.jpg', '0_4174046.png', '0_4048869.png', '0_4053325.jpg', '0_4090332.png']
path = "D:/2 UserStudy/Case3/landing_test"
new_path = "D:/2 UserStudy/Case3/result"

for root, dirs, files in os.walk(path):     
    for i in range(len(positive)):
        file_path = root+'/' + positive[i]
        new_file_path = new_path+ '/' + positive[i]
        shutil.copy(file_path, new_file_path)

In [40]:
# 获取所有的id
path = "D:/2 UserStudy/Case3/result"
original_path = "D:/2 UserStudy/Case3/x"
new_path = "D:/2 UserStudy/Case3/y"
fnames = []
for file in os.listdir(path):
    fname = file[2:]
    fnames.append(fname)
    
for root, dirs, files in os.walk(original_path):
    for i in range(len(fnames)):
        file_path = root+'/'+fnames[i]
        new_file_path = new_path+"/"+fnames[i]
        shutil.copy(file_path, new_file_path)

## CASE 4: mobile + finance + chart

In [3]:
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'web design', 'movie', 'computer', 'laptop', "webdesign", "facebook", "twitter", "blogging", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]

In [39]:
# case 4:  mobile + finance + chart
ios = ["ios", "ios11", "ios_11", "ios 11","ios_app"]
class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] == "chart":
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] == "finance":
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] in ios:
                                    class_0.append(df.values[i,0])
                                    break

print(len(class_0))
print(len(set(class_0)))

13
4


In [40]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case4"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        c = os.path.isfile(root + '/' + class_0[i] + ".gif")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_0[i] + ".gif"
            new_file_path = new_path+ '/' + class_0[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [6]:
mobile = ["mobile", "ios","phone","iphone","iphonex","iphone 5", "iphone 6", "iphone 7", "ios11", "smartphone","smartphone 2xl", 'android',"iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","mobile app iphone", "ipad_pro","android_app","mobile_application","mobile app","mobileapp", "mobile", "mobile design"]
finance = ["bank", "bitcoin","blockchain", "cryptocurrency", "crypto exchange", "crypto wallet", "finance", "banking", "bank app", "banking app", "fintech", "finances","finance_app","finance app","finance application","finance_app","finance_application","banking app","banding_app","bank application","bank app","bank_app","financial","investing", "wallet"]
chart = ["chart", "charts", "diagram", "statistic","statics", "data analysis", "data visualization", "graph", "dashboard", "circular chart","pie chart"]

In [7]:
class_1 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in mobile:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in finance:
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] in chart:
                                    class_1.append(df.values[i,0])
                                    break

print(len(class_1))

594


In [8]:
print(len(set(class_1)))

115


In [9]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case4/more search"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        c = os.path.isfile(root + '/' + class_1[i] + ".gif")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_1[i] + ".gif"
            new_file_path = new_path+ '/' + class_1[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [14]:
class_2 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in finance:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in chart:
                            class_2.append(df.values[i,0])
                            break
print(len(class_2))
print(len(set(class_2)))

866
306


In [15]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case4/finance+chart"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [52]:
path = "D:/2 UserStudy/Case4/x_2"
new_path = "D:/2 UserStudy/Case4/chart_test"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

In [53]:
import os
path_0 = "D:/2 UserStudy/Case4/chart_test"
for file in os.listdir(path_0):
    os.rename(os.path.join(path_0,file),os.path.join(path_0,"0_"+file))

In [54]:
positive = ['0_4144011.png', '0_4086235.png', '0_4258813.jpg', '0_4111612.jpg', '0_4159111.png', '0_4213571.png', '0_4258865.png', '0_4048525.png', '0_4046895.jpg', '0_4303214.png', '0_4066410.jpg', '0_4290131.png', '0_4217157.png', '0_4172671.png', '0_4329654.jpg', '0_4236115.png', '0_4316646.png', '0_4042460.png', '0_4304506.jpg', '0_4114099.jpg', '0_4088965.png', '0_4108581.png', '0_4271031.png', '0_4025947.png', '0_4017001.png', '0_4116164.png', '0_4032309.jpg', '0_4163925.jpg', '0_4239498.png', '0_4003612.png', '0_4173291.png', '0_4172595.jpg', '0_4145160.jpg', '0_4288234.png', '0_4302539.png', '0_4050190.png', '0_4295606.png', '0_4243608.jpg', '0_4273870.png', '0_4007167.jpg', '0_4006648.jpg', '0_4075832.png', '0_4151289.jpg', '0_4206928.png', '0_4049947.png', '0_4271157.png', '0_4283010.png', '0_4055887.jpg', '0_4302910.png', '0_4215413.png', '0_4200133.png', '0_4116315.jpg', '0_4025368.png', '0_4283631.png', '0_4204637.png', '0_4099818.png', '0_4034367.jpg', '0_4288463.png', '0_4016531.png', '0_4323058.png', '0_4094110.png', '0_4020480.jpg', '0_4275945.png', '0_4315083.png', '0_4219204.png', '0_4029121.png', '0_4192757.png', '0_4258538.png', '0_4216485.jpg', '0_4046203.jpg', '0_4287106.png', '0_4021878.jpg', '0_4258867.png', '0_4003062.png', '0_4070793.png', '0_4222952.jpg', '0_4286025.jpg', '0_4301862.png', '0_4329636.png', '0_4081002.jpg', '0_4130853.jpg', '0_4051288.jpg', '0_4133531.png', '0_4166995.jpg', '0_4301827.png', '0_4074800.png', '0_4194704.png', '0_4282129.png', '0_4070929.jpg', '0_4301305.png', '0_4044517.jpg', '0_4320425.png', '0_4104885.png', '0_4082967.png', '0_4115808.jpg', '0_4294254.jpg', '0_4024044.png', '0_4003111.png', '0_4264115.jpg', '0_4053995.png', '0_4048523.jpg', '0_4147546.jpg', '0_4073921.jpg', '0_4009733.png', '0_4082380.png', '0_4151329.jpg', '0_4240230.jpg', '0_4287491.png', '0_4284206.jpg', '0_4178165.png', '0_4274725.png', '0_4302703.png', '0_4247522.png', '0_4019199.png', '0_4135322.jpg', '0_4013292.jpg', '0_4007164.jpg', '0_4055834.jpg', '0_4206210.png', '0_4198312.jpg', '0_4250611.jpg', '0_4078728.png', '0_4157753.jpg', '0_4025901.png', '0_4032043.png', '0_4328285.png', '0_4285553.jpg', '0_4096471.png', '0_4042993.png', '0_4070662.png', '0_4184568.png', '0_4003743.jpg', '0_4324662.jpg', '0_4313624.png', '0_4053623.png', '0_4121101.png', '0_4070440.png', '0_4014775.jpg', '0_4080389.png', '0_4257005.png', '0_4212110.png', '0_4101856.png', '0_4269680.png', '0_4328051.png', '0_4168402.jpg', '0_4246226.png', '0_4253671.png', '0_4197296.jpg', '0_4058234.png', '0_4236902.png', '0_4046711.jpg', '0_4051346.png', '0_4263355.png', '0_4149018.png', '0_4034719.jpg', '0_4055460.png', '0_4294383.png', '0_4006412.jpg', '0_4251084.jpg', '0_4038618.png', '0_4312964.jpg', '0_4277937.png', '0_4003443.png', '0_4049332.png', '0_4282463.jpg', '0_4106536.png', '0_4258862.png']
path = "D:/2 UserStudy/Case4/chart_test"
new_path = "D:/2 UserStudy/Case4/result_1"

for root, dirs, files in os.walk(path):     
    for i in range(len(positive)):
        file_path = root+'/' + positive[i]
        new_file_path = new_path+ '/' + positive[i]
        shutil.copy(file_path, new_file_path)

In [55]:
positive = ['0_4144011.png', '0_4086235.png', '0_4303978.jpg', '0_4159111.png', '0_4213571.png', '0_4052202.jpg', '0_4064956.png', '0_4201172.png', '0_4237397.png', '0_4027779.png', '0_4072030.jpg', '0_4290131.png', '0_4217157.png', '0_4187655.png', '0_4056886.jpg', '0_4329654.jpg', '0_4316646.png', '0_4042460.png', '0_4304506.jpg', '0_4202975.png', '0_4088965.png', '0_4156704.jpg', '0_4271031.png', '0_4025947.png', '0_4117723.png', '0_4299499.jpg', '0_4017001.png', '0_4327771.png', '0_4014910.jpg', '0_4199834.png', '0_4121303.png', '0_4172595.jpg', '0_4221950.png', '0_4010736.jpg', '0_4055860.jpg', '0_4302539.png', '0_4089742.png', '0_4050190.png', '0_4088302.png', '0_4256488.png', '0_4318469.jpg', '0_4075832.png', '0_4071699.jpg', '0_4136095.png', '0_4049947.png', '0_4271157.png', '0_4283010.png', '0_4082342.jpg', '0_4055887.jpg', '0_4244662.jpg', '0_4131091.png', '0_4200133.png', '0_4283631.png', '0_4204637.png', '0_4034367.jpg', '0_4050459.png', '0_4016531.png', '0_4268639.png', '0_4094110.png', '0_4020480.jpg', '0_4219204.png', '0_4192757.png', '0_4258538.png', '0_4046203.jpg', '0_4114355.png', '0_4057078.png', '0_4048477.png', '0_4003062.png', '0_4222952.jpg', '0_4059672.png', '0_4204347.jpg', '0_4242110.png', '0_4301862.png', '0_4246604.jpg', '0_4081002.jpg', '0_4122858.png', '0_4166995.jpg', '0_4135413.png', '0_4074800.png', '0_4194704.png', '0_4070929.jpg', '0_4005070.png', '0_4044517.jpg', '0_4104885.png', '0_4082967.png', '0_4115808.jpg', '0_4294254.jpg', '0_4003111.png', '0_4294312.jpg', '0_4048523.jpg', '0_4297018.jpg', '0_4073921.jpg', '0_4082380.png', '0_4268049.jpg', '0_4240230.jpg', '0_4127710.png', '0_4284206.jpg', '0_4247522.png', '0_4013292.jpg', '0_4194076.jpg', '0_4225099.png', '0_4136442.png', '0_4055834.jpg', '0_4198312.jpg', '0_4321730.png', '0_4250611.jpg', '0_4078728.png', '0_4187380.png', '0_4025901.png', '0_4328285.png', '0_4285553.jpg', '0_4096471.png', '0_4184568.png', '0_4184217.png', '0_4324662.jpg', '0_4057331.png', '0_4096828.jpg', '0_4121101.png', '0_4070440.png', '0_4014775.jpg', '0_4080389.png', '0_4012731.jpg', '0_4257005.png', '0_4101856.png', '0_4328051.png', '0_4198346.jpg', '0_4197296.jpg', '0_4058234.png', '0_4046711.jpg', '0_4051346.png', '0_4164480.png', '0_4263355.png', '0_4014583.png', '0_4034719.jpg', '0_4108787.png', '0_4177922.png', '0_4055460.png', '0_4294383.png', '0_4038618.png', '0_4277937.png', '0_4049332.png', '0_4282463.jpg', '0_4106536.png']
path = "D:/2 UserStudy/Case4/chart_test"
new_path = "D:/2 UserStudy/Case4/result_2"

for root, dirs, files in os.walk(path):     
    for i in range(len(positive)):
        file_path = root+'/' + positive[i]
        new_file_path = new_path+ '/' + positive[i]
        shutil.copy(file_path, new_file_path)

In [56]:
# 获取所有的id
path = "D:/2 UserStudy/Case4/result"
original_path = "D:/2 UserStudy/Case4/x"
new_path = "D:/2 UserStudy/Case4/y"
fnames = []
for file in os.listdir(path):
    fname = file[2:]
    fnames.append(fname)
    
for root, dirs, files in os.walk(original_path):
    for i in range(len(fnames)):
        file_path = root+'/'+fnames[i]
        new_file_path = new_path+"/"+fnames[i]
        shutil.copy(file_path, new_file_path)

## CASE 5: website+login+form

In [27]:
# case 5:  website+login+form

class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] == "form":
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] == "login":
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] == "web":
                                    class_0.append(df.values[i,0])
                                    break

print(len(class_0))
print(len(set(class_0)))

42
12


In [28]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case5"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        #c = os.path.isfile(root + '/' + class_0[i] + ".gif")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)
#         if c:
#             file_path = root+'/' + class_0[i] + ".gif"
#             new_file_path = new_path+ '/' + class_0[i] + ".gif"
#             shutil.copy(file_path,new_file_path)

In [39]:
web = ["website","webdesigner", "web-design", "design_for_website","websitedesign", "web app","webpage", "website", "web", "website design","webpage design"]
signup = ["sign up","signup","sign_up","login","log in","log_in", "signin", "sign in", "sign_in", "logup", "log up", "log_up"]
form = ["form", "form design", "signupform","signup form","sign up page","signup page","application form"]


In [40]:
class_1 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in web:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in signup:
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] in form:
                                    class_1.append(df.values[i,0])
                                    break

print(len(class_1))
print(len(set(class_1)))

175
33


In [41]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case5/more search"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        c = os.path.isfile(root + '/' + class_1[i] + ".gif")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_1[i] + ".gif"
            new_file_path = new_path+ '/' + class_1[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [46]:
class_2 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in web:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in form:
                            class_2.append(df.values[i,0])
                            break
print(len(class_2))
print(len(set(class_2)))

509
175


In [47]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case5/web+form"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [48]:
#所有website的图片
class_2 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in web:
                        class_2.append(df.values[i,0])
                        break
print(len(class_2))
print(len(set(class_2)))

44432
17619


In [49]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case5/web"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)

In [51]:
path = "D:/2 UserStudy/Case5/web"
new_path = "D:/2 UserStudy/Case5/web_test"

for fname in  os.listdir(path):
    fpath = os.path.join(path, fname)
    img=Image.open(fpath)
    new_img=img.resize((256, 256),Image.BILINEAR) 
    new_img = new_img.convert("RGB")

    new_img.save(os.path.join(new_path,fname))

C:\Users\DELL\Anaconda3\lib\site-packages\PIL\Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


OSError: cannot identify image file 'D:/2 UserStudy/Case5/web\\4228529.jpg'

In [2]:
import os
path_0 = "D:/2 UserStudy/Case5/web_test"
for file in os.listdir(path_0):
    os.rename(os.path.join(path_0,file),os.path.join(path_0,"0_"+file))

In [11]:
positive = ['0_4026678.png', '0_4028569.png', '0_4035678.jpg', '0_4038383.png', '0_4062256.png', '0_4022755.png', '0_4055534.png', '0_4040484.jpg', '0_4022652.jpg', '0_4037799.png', '0_4049350.png', '0_4027379.png', '0_4027446.png', '0_4044185.jpg', '0_4037829.jpg', '0_4023742.jpg', '0_4057635.png', '0_4063315.png', '0_4046440.png', '0_4053318.png', '0_4023073.jpg', '0_4037386.png', '0_4041097.jpg', '0_4027293.png', '0_4043646.png', '0_4035249.png', '0_4034641.jpg', '0_4026032.png', '0_4023202.png', '0_4062399.jpg', '0_4031734.png', '0_4038164.jpg', '0_4034876.jpg', '0_4026976.png', '0_4027943.png', '0_4040747.jpg', '0_4030857.jpg', '0_4049915.png', '0_4040113.jpg', '0_4058054.jpg', '0_4051927.png', '0_4033146.jpg', '0_4060089.png', '0_4051834.jpg', '0_4055068.png', '0_4052193.png', '0_4028065.png', '0_4060059.png', '0_4036839.jpg', '0_4050549.png', '0_4030388.png', '0_4032864.jpg', '0_4029454.jpg', '0_4050632.png', '0_4045943.png', '0_4031290.jpg', '0_4041152.png', '0_4027582.png', '0_4059006.png', '0_4052502.jpg', '0_4048101.png', '0_4049426.jpg', '0_4037023.jpg', '0_4059080.png', '0_4053194.png', '0_4040896.png', '0_4037367.png', '0_4051294.png', '0_4048025.png', '0_4037369.png', '0_4056782.jpg', '0_4037244.png', '0_4061321.jpg', '0_4041716.jpg', '0_4061939.png', '0_4060956.jpg', '0_4030910.jpg', '0_4048566.jpg', '0_4044618.jpg', '0_4049270.png', '0_4045630.jpg', '0_4029685.png', '0_4032491.png', '0_4036149.jpg', '0_4032908.png', '0_4058808.png', '0_4062013.png', '0_4048944.png', '0_4031093.png', '0_4037939.jpg', '0_4061851.jpg', '0_4063912.png', '0_4037089.png', '0_4050744.png', '0_4057245.jpg', '0_4041074.png', '0_4045726.png', '0_4026600.jpg', '0_4028779.jpg', '0_4061855.png', '0_4059909.png', '0_4061958.png', '0_4058492.jpg', '0_4049276.png', '0_4059780.jpg', '0_4028853.jpg', '0_4041707.jpg', '0_4060121.png', '0_4045600.jpg', '0_4031733.jpg', '0_4040124.png', '0_4056059.jpg', '0_4046605.jpg', '0_4039071.png', '0_4026265.jpg', '0_4058375.png']

path = "D:/2 UserStudy/Case5/1"
new_path = "D:/2 UserStudy/Case5/x"

for root, dirs, files in os.walk(path):     
    for i in range(len(positive)):
        file_path = root+'/' + positive[i]
        new_file_path = new_path+ '/' + positive[i]
        shutil.copy(file_path, new_file_path)

In [12]:
# 获取所有的id
path = "D:/2 UserStudy/Case5/x"
original_path = "D:/2 UserStudy/Case5/web"
new_path = "D:/2 UserStudy/Case5/y"
fnames = []
for file in os.listdir(path):
    fname = file[2:]
    fnames.append(fname)
    
for root, dirs, files in os.walk(original_path):
    for i in range(len(fnames)):
        file_path = root+'/'+fnames[i]
        new_file_path = new_path+"/"+fnames[i]
        shutil.copy(file_path, new_file_path)

## CASE 6: website+landing page+blue


In [18]:
keywords = ["smartphone", "google", "smartphone 2xl", "blog", "homepage", "landingpage", "css", "html",'wordpress', "javascript", 'android', 'iphone x','card payments', 'clean ui', 'career service website', 'mobile-first design', 'ux design', 'ux research', 'web design', 'movie', 'computer', 'laptop', "webdesign", "facebook", "twitter", "blogging", "profile", "social platform", "ui","user interface", "user-interface", "user_interface", "user interface design", "user_interface_design", "user-interface-design", "uidesign", "ui design", "ui-design", "ui_design", "uiuxdesign", "uxuidesign", "uiux design", "uxui design", "uiux_design", "uxui_design", "uiux-design", "uxui-design", "uiux", "uidesigner", "ui_ux", "ui.ux", "ux.ui", "uxui", "ui-ux", "ux-ui", "app-ui", "daily ui", "dailyui", "daily_ui", "30_days_of_ui", "30 days of ui", "iphone", "iphonex","iphone_x", "iphone8", "iphone7", "ios_11","iphone_app","ipad_pro", "travelapp", "ios11", "website_mockup", "website_template", "landing page","app", "landing", "application", "android app", "app-design", "website", "app design", "webdesigner", "web-design", "design_for_website", "app_design", "appdesign", "websitedesign", "web app", "webapp", "ios app", "ios_app", "ios_design", "android_app", "app_development", "mobile_application", "application_design", "mobile app","mobileapp", "mobile", "mobile design", "mobiledesign","webpage", "website", "web", "website design","webpage design", "login", "signup", "ecommerce", "e commerce", "e-commerce"]
web = ["website","webdesigner", "web-design", "design_for_website","websitedesign", "web app","webpage", "website", "web", "website design","webpage design"]
blue=["blue"]
landing = ["landing page", "landing pages","landingpage","landingpages","landing_pages","landing_page"]
music = ["music", "music player", "musicplayer", "music_player","music-player","music players","musicplayers","music_players","apple_music", "apple music","apple-music","playlist", "music app","musicapp","music_app", "music application","music_applications", "music applications","music player app","music player ui"]

In [29]:
# case 5:  website+landingpage+blue
class_0 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] == "web design":
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] == "landing page":
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] == "blue":
                                    class_0.append(df.values[i,0])
                                    break

print(len(class_0))
print(len(set(class_0)))

16
4


In [17]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case6/Base_landing"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_0)):
        a = os.path.isfile(root + '/' + class_0[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_0[i] + ".png")
        c = os.path.isfile(root + '/' + class_0[i] + ".gif")
        if a:
            file_path = root+'/' + class_0[i] + ".jpg"
            new_file_path = new_path+ '/' + class_0[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_0[i] + ".png"
            new_file_path = new_path+ '/' + class_0[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_0[i] + ".gif"
            new_file_path = new_path+ '/' + class_0[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [20]:
class_1 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in web:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in blue:
                            for y in range(0, df.shape[1]):
                                if df.values[i, y] in landing:
                                    class_1.append(df.values[i,0])
                                    break

print(len(class_1))
print(len(set(class_1)))

112
24


In [21]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case6/more search"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_1)):
        a = os.path.isfile(root + '/' + class_1[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_1[i] + ".png")
        c = os.path.isfile(root + '/' + class_1[i] + ".gif")
        if a:
            file_path = root+'/' + class_1[i] + ".jpg"
            new_file_path = new_path+ '/' + class_1[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_1[i] + ".png"
            new_file_path = new_path+ '/' + class_1[i] + ".png"
            shutil.copy(file_path,new_file_path)
        if c:
            file_path = root+'/' + class_1[i] + ".gif"
            new_file_path = new_path+ '/' + class_1[i] + ".gif"
            shutil.copy(file_path,new_file_path)

In [29]:
class_2 = []
for i in range(0, len(df)):
    for j in range(0, df.shape[1]):
        if df.values[i,j] in keywords:
            for k in range(0, df.shape[1]):
                if df.values[i,k] in web:
                    for x in range(0, df.shape[1]):
                        if df.values[i, x] in landing:
                            class_2.append(df.values[i,0])
                            break
print(len(class_2))
print(len(set(class_2)))

8380
1729


In [26]:
path = 'D:/0 PROJECT/dribbble-final version/All_images'
new_path = "D:/2 UserStudy/Case6/landing+web"

for root, dirs, files in os.walk(path):     
    for i in range(len(class_2)):
        a = os.path.isfile(root + '/' + class_2[i] + ".jpg")
        b = os.path.isfile(root + '/' + class_2[i] + ".png")
        
        if a:
            file_path = root+'/' + class_2[i] + ".jpg"
            new_file_path = new_path+ '/' + class_2[i] + ".jpg"
            shutil.copy(file_path, new_file_path)
        if b:
            file_path = root+'/' + class_2[i] + ".png"
            new_file_path = new_path+ '/' + class_2[i] + ".png"
            shutil.copy(file_path,new_file_path)
